In [78]:
import os
import pandas as pd

In [79]:
#directory = "/home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks/data/repeat05/"

In [91]:
# Get the directory with the notebooks, including its subdirectories
current_dir = os.getcwd()

# Get the directories with data and results
data_dir = os.path.join(current_dir, "data")
temporary_dir= os.path.join(current_dir, "temporary_data")
results_dir = os.path.join(current_dir, "results")

project_dir = os.path.dirname(current_dir)

print(project_dir, current_dir, results_dir, data_dir)

/home/xternava/Documents/GitHub/bloat-energy-consumption /home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks /home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks/results /home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks/data


In [81]:
# Using this function, the purpose is to cancatenate all generated measurements for each repeated experiment 
def concatenate_csv_files(directory_path, output_file):
    file_list = os.listdir(directory_path)
    csv_files = [file for file in file_list if file.endswith(".csv")]

    if not csv_files:
        print("No CSV files found in the directory.")
        return

    dfs = []
    for file in csv_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path, delimiter=';') # This is important, the delimiter
        filename_parts = file[:-4].split("_")  # Split filename and remove ".csv" extension
        df['Program'] = filename_parts[0]
        df['TestNr'] = filename_parts[1]
        df['Utilities'] = filename_parts[2]
        dfs.append(df)

    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df.to_csv(output_file, index=False, sep=';')  # Use semicolon as delimiter

    print(f"CSV files concatenated and saved to '{output_file}'.")

# All data into files in the folder save in a new CSV file
input_data = os.path.join(data_dir, 'repeat01/') # You need to change this for each new folder that you want to concatenate its files
output_data = os.path.join(results_dir, 'cat_repeat01.csv') # You need to change this for each new folder given above

concatenate_csv_files(input_data, output_data)

CSV files concatenated and saved to '/home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks/results/cat_repeat01.csv'.


In [87]:
# Using this function, the purpose is to concatenate all resulting files from all 5 executions
def concatenate_allcsv_files(directory_path, output_file):
    file_list = os.listdir(directory_path)
    csv_files = [file for file in file_list if file.endswith(".csv")]

    if not csv_files:
        print("No CSV files found in the directory.")
        return

    dfs = []
    for file in csv_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path, delimiter=';') # This is important, the delimiter
        df['Repetition'] = file  # Add filename as a new column
        dfs.append(df)

    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df.to_csv(output_file, index=False, sep=';')  # Use semicolon as delimiter


    print(f"CSV files concatenated and saved to '{output_file}'.")

# Save in a new CSV file
output_alldata = os.path.join(results_dir, 'cat_all5repeats.csv')

concatenate_allcsv_files(temporary_dir, output_alldata)

CSV files concatenated and saved to '/home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks/results/cat_all5repeats.csv'.


In [88]:
# Now we need to find the average value of PSYS for all measurements of a program 
def calculate_average_PSYS(input_file, output_file):
    # Read the CSV file
    df = pd.read_csv(input_file, delimiter=';')
    
    # Calculate average of 'PSYS' column for each unique combination of 'Program' and 'Utilities'
    average_psys = df.groupby(['Program', 'Utilities'])['PSYS'].mean().reset_index()
    
    # Write the average values to a new file separated with a semicolon
    average_psys.to_csv(output_file, index=False, sep=';')
    
    print(f"Average PSYS values written to '{output_file}'.")

# Required inputs and outputs
input_data = os.path.join(results_dir, 'cat_all5repeats.csv')
output_data = os.path.join(results_dir, 'averagePSYS_all5repeats.csv')

calculate_average_PSYS(input_data, output_data)


Average PSYS values written to '/home/xternava/Documents/GitHub/bloat-energy-consumption/notebooks/results/averagePSYS_all5repeats.csv'.


In [95]:
# After we need to put together the PSYS and Size values for each program in each of the 3 different implementations
def merge_csv_files(first_csv_path, second_csv_path, output_csv_path):
    # Read the first CSV file into a DataFrame
    first_df = pd.read_csv(first_csv_path, delimiter=';')

    # Read the second CSV file into another DataFrame
    second_df = pd.read_csv(second_csv_path, delimiter=';')

    # Merge the two DataFrames base on the 'Program' column
    merged_df = pd.merge(second_df, first_df, on='Program')
    
    # Reorder the columns, we want to have 'Size' at the end
    columns = list(merged_df.columns)
    columns.remove('Size')
    columns.append('Size')
    merged_df = merged_df[columns]
    

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_csv_path, index=False)

# Example usage
csv_with_ec = os.path.join(results_dir, 'averagePSYS_all5repeats.csv')
csv_with_size = os.path.join(project_dir, 'pre-experiment/res-bs-busybox.csv') # This is wrong, but still I'm fixing it
output_file = os.path.join(results_dir, 'all_ec_bsize.csv')

merge_csv_files(csv_with_ec, csv_with_size, output_file)
